# Red Neuronal con Theano

En la implementación computacional de redes neuronales, es común trabajar con grandes cantidades de datos en busca de la convergencia del método. Sin embargo, esto representa un gasto computacional considerable.

La librería de theano permite optimizar este proceso. Esta librería ayuda a definir las expresiones matemáticas de una manera sencilla y al mismo tiempo optimiza el manejo de meoria cuando se trabajo con arreglos multi-dimensionales.

Más sobre esta librería puede encontrarse en:

http://deeplearning.net/software/theano/

y en:

https://arxiv.org/pdf/1605.02688.pdf

La siguiente implementación en theano de una Feedforward Neural Network yoma como argumento los datos de ejmeplo (ex) y las etiquetas del supervisor (lab). Asimismo, diferentes parámetros pueden ser configurados: número de unidades ocultas, número de iteraciones (it), parámetro $\eta$ (rate), y parámetro de regularización (reg).

In [2]:
import numpy as np
import theano.tensor as T
from theano import *


def NN(ex,lab,hidden=1, its=1000, rate=0.01, reg=1):
    #Se declaran las variables con que se trabajará
    X,Y = T.dmatrices('X','Y')
    y = T.dvector('y')
    
    #El total de clases que arrojará
    out_dims= len(set(lab))
    #El número de rasgos que representan cada vector
    input_dims = ex.shape[1]
    
    #Con estos datos se generan los vectores de parámetros
    np.random.seed(0)
    W1 = shared(np.random.randn(input_dims, hidden)/np.sqrt(input_dims), name='W1' )
    b1 = shared(np.zeros(hidden), name='b1')
    W2 = shared(np.random.randn(hidden, out_dims)/np.sqrt(hidden), name='W2' )
    b2 = shared(np.zeros(out_dims), name='b2')

    #Defino la función Wx+b y la función de activación g=tanh
    f1, updates = scan(lambda v: T.dot(v,W1) + b1, sequences=X )
    g = T.tanh(f1)
    f2, updates = scan(lambda v: T.dot(v,W2) + b2, sequences=g )
    
    #Uso la función Softmax para determinar las probabilidades de las clases
    exp_scores = T.exp(f2)
    prob = exp_scores / T.sum(exp_scores, axis=1, keepdims=True)
    get_probs = function(inputs=[X], outputs=[prob,g])

    #Defino la función de predicción que arrojará la clase con mayor probabilidad
    predict = function(inputs=[X], outputs=[T.argmax(prob,axis=1)])

    #Determinó las derivadas de mis funciones
    dW2 = T.dot(Y.T,X)
    db2 = T.sum(X, axis=0, keepdims=True)
    d2 = T.dot(X, W2.T) * (1- Y**2)
    dW1 = T.dot(ex.T, d2)
    db1 = T.sum(d2, axis=0)

    #Actualizo los parámetros de aprendizaje por medio de Stochastic Gradient Descent (SGD)
    train = function(inputs=[X,Y], outputs=[dW1,db1,dW2,db2], updates=[(W2, W2-rate*(dW2 + reg*dW2)), (b2, b2-rate*db2[0]), (W1, W1-rate*(dW1 + reg*dW1)), (b1, b1-rate*db1) ])

    #Itero el algoritmo
    for i in range(0,its):
        probs, activation = get_probs(ex)
        probs[range(len(ex)), lab] -= 1
        train(probs, activation)

    return  predict, (W1.get_value(),b1.get_value(),W2.get_value(),b2.get_value())

Ahora podemos evaluar unos datos de ejmplo a partir de la red neuronal. En este caso, nos arrojará la función de predicción y el modelo aprendido.

In [5]:
X = np.array([[0,1],[1,0],[0,0],[1,1]])
y = np.array([0,0,1,1])

predict, model = NN(X,y,3)

print model

(array([[ 2.44863591, -1.31374088,  2.65196629],
       [ 2.45817634,  2.67103475, -1.58839646]]), array([-0.54382113,  0.19661476,  0.48807415]), array([[ 2.42088631, -1.95973863],
       [-1.98607969,  2.16354542],
       [-1.62913443,  2.55192323]]), array([ 0.02312234, -0.02312234]))


Con la función de predicción obtenida, podemos evaluar nuestros datos.

In [6]:
z = [[0,0.2]]
print predict(z)

[array([1])]
